In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from datetime import timedelta
import sqlalchemy import create_engine

# ----- Load market history CSVs -----
csv_dir = os.path.abspath('../output/market_history')

market_df = pd.concat([
    pd.read_csv(os.path.join(csv_dir, f'market_history_batch_{i}.csv'))
    for i in range(1, 49)
    if os.path.exists(os.path.join(csv_dir, f'market_history_batch_{i}.csv'))
], ignore_index=True)

market_df['date'] = pd.to_datetime(market_df['date'])

# ----- Load item names from PostgreSQL -----
conn = psycopg2.connect(
    dbname="eve_data",
    user="postgres",
    password="109009885",  # replace if needed
    host="localhost",
    port="5432"
)
item_map = pd.read_sql_query("SELECT type_id, type_name FROM inv_types", conn)
item_map = item_map.rename(columns={'type_name': 'item_name'})
conn.close()

# ----- Merge market data with item names -----
full_df = market_df.merge(item_map, on='type_id')

# ----- Set up dropdown and plot widget -----
item_names = sorted(full_df['item_name'].unique())
dropdown = widgets.Combobox(
    placeholder='Search item...',
    options=item_names,
    description='Item:',
    ensure_option=True,
    layout=widgets.Layout(width='50%')
)

output = widgets.Output()

def plot_item(change):
    output.clear_output()
    selected = change['new']
    if selected not in item_names:
        return
    with output:
        df = full_df[
            (full_df['item_name'] == selected) &
            (full_df['date'] >= pd.Timestamp.today() - timedelta(days=365))
        ]
        if df.empty:
            print("No data in last year.")
            return
        plt.figure(figsize=(10, 5))
        plt.plot(df['date'], df['average'], label='Average Price')
        plt.title(f"{selected} — Last 365 Days")
        plt.xlabel('Date')
        plt.ylabel('ISK')
        plt.grid()
        plt.legend()
        plt.tight_layout()
        plt.show()

dropdown.observe(plot_item, names='value')
display(dropdown, output)


C:\Users\Jouke\AppData\Local\Temp\ipykernel_33760\2248263563.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  item_map = pd.read_sql_query("SELECT type_id, type_name FROM inv_types", conn)


Combobox(value='', description='Item:', ensure_option=True, layout=Layout(width='50%'), options=(' Tyrant Blue…

Output()